In [31]:
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold 
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split



device = 'cpu'

#Code loosly based on https://datascience.stackexchange.com/questions/56804/sckit-learn-cross-validation-and-model-retrain
#And our exercise 1a

In [2]:
train = pd.read_csv(r'C:\Users\erics\Documents\Programme\IntroML\Task1\task1b_ql4jfi6af0\train.csv')

X = train.iloc[:,2:]
Y = train.iloc[:,1]
ID = train.iloc[:,0]
pd.set_option("display.precision", 7)
print(X)
print(Y)
print(X.shape)
print(Y.shape)

       x1    x2    x3    x4    x5
0    0.02  0.05 -0.09 -0.43 -0.08
1   -0.13  0.11 -0.08 -0.29 -0.03
2    0.08  0.06 -0.07 -0.41 -0.03
3    0.02 -0.12  0.01 -0.43 -0.02
4   -0.14 -0.12 -0.08 -0.02 -0.08
..    ...   ...   ...   ...   ...
695  0.31 -0.09  0.04 -0.09  0.03
696 -0.26 -0.01  0.02 -0.40  0.05
697 -0.27 -0.22 -0.01 -0.32 -0.05
698  0.19  0.11 -0.05 -0.27 -0.04
699 -0.09 -0.09 -0.06 -0.41 -0.06

[700 rows x 5 columns]
0     -6.8226793
1     -6.3262905
2     -9.3027282
3     -7.3718925
4     -6.0276470
         ...    
695   -6.1473937
696   -6.2148989
697   -6.8413411
698   -6.5233707
699   -5.6766827
Name: y, Length: 700, dtype: float64
(700, 5)
(700,)


Wir verlieren etwas präzision durch das Laden in den DataFrame (durch float precision)

In [3]:
def transform(X):
    l1 = X
    l2 = X*X
    l3 = np.exp(X)
    l4 = np.cos(X)
    const = np.ones((X.shape[0],1))
    return np.concatenate((l1, l2, l3, l4, const), axis=1)

In [4]:
transformer = FunctionTransformer(transform, validate=True)

In [5]:
#X_Test = np.array([[1, 2, 3, 4, 5], [1, 1,1, 1,1], [1, 1,1, 1,1], [1, 1,1, 1,1]])
#print(transformer.transform(X_Test))

In [10]:
X_trans = transformer.transform(X)
print(X_trans.shape)
#print(X_trans)

(700, 21)


In [11]:
param_grid = {'alpha' : np.logspace(-2, 10, num=200)}
kf = KFold(n_splits=10)
model = Ridge(alpha=1)

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring='neg_root_mean_squared_error', return_train_score=True)

In [12]:
best_model = grid_search.fit(X_trans, Y)
print("Lowest RMSE:" + str(best_model.best_score_))
best_alpha = best_model.best_estimator_
print("With alpha: " + str(best_alpha))

coefficiants = best_model.best_estimator_.coef_
print("And coefficiants. ")
print(coefficiants)

Lowest RMSE:-1.9544619103436798
With alpha: Ridge(alpha=5.170920242896756)
And coefficiants. 
[-0.06612788 -0.68231224 -0.83384997 -0.05459609 -0.04157483 -0.24778276
  0.19426473  0.16548507 -0.03504243  0.10998607 -0.19867299 -0.5892195
 -0.75919001 -0.08069978  0.00312958  0.12313297 -0.0966012  -0.08219195
  0.01657758 -0.05327217  0.        ]


store


In [ ]:
dictionary = {'Coefficiants': coefficiants}
avg_acc_df = pd.DataFrame(dictionary )
print(avg_acc_df)
avg_acc_df.to_csv(r'C:\Users\erics\Documents\Programme\IntroML\Task1\submission1b.csv', index=False, header=False)

In [14]:
model2 = Ridge(alpha=5.170920242896756)
best_fit = model2.fit(X_trans, Y)
coef = best_fit.coef_
print(coef)

[-0.06612788 -0.68231224 -0.83384997 -0.05459609 -0.04157483 -0.24778276
  0.19426473  0.16548507 -0.03504243  0.10998607 -0.19867299 -0.5892195
 -0.75919001 -0.08069978  0.00312958  0.12313297 -0.0966012  -0.08219195
  0.01657758 -0.05327217  0.        ]


Overfitting Data This does not work

In [17]:
model3 = LinearRegression()
best_fit = model3.fit(X_trans, Y)
coef = best_fit.coef_
print(coef)

[   87.19891672   -67.27010868    28.53822965   158.07550392
  -819.69642957   200.04958807  -393.54175642  1953.26216011
  -523.63288817  1518.93422167   -86.60565699    65.34027498
   -30.9921308   -155.77167729   822.10809668   327.109988
  -737.64020339  3890.67031218 -1211.22831197  3799.82175151
     0.        ]


Versuche nochmals 10 fold cross validation

In [24]:
k = 10
kf = KFold(n_splits=k, random_state=None)
model4 = Ridge(alpha=5.170920242896756)
acc_score = []
best_acc = 1000


In [27]:
for train_index , test_index in kf.split(X):
        X_train , X_test = X_trans[train_index,:],X_trans[test_index,:]
        Y_train , Y_test = Y[train_index] , Y[test_index]

        model4.fit(X_train,Y_train)
        pred_values = model4.predict(X_test)

        acc = np.sqrt(mean_squared_error(pred_values , Y_test))
        if acc < best_acc:
            best_acc = acc
            best_model = model4
            
        acc_score.append(acc)

In [28]:
print(best_acc)
print(acc_score)

1.432199870633657
[1.8485391074358133, 1.87281313622276, 2.058194759256672, 2.031439206673872, 1.432199870633657, 2.2027697621467093, 2.083450831416385, 1.9344169064790657, 2.087398928980819, 1.993396594191042, 1.8485391074358133, 1.87281313622276, 2.058194759256672, 2.031439206673872, 1.432199870633657, 2.2027697621467093, 2.083450831416385, 1.9344169064790657, 2.087398928980819, 1.993396594191042]


In [29]:
print(model4.coef_)

[-0.08763883 -0.7148755  -0.83292098 -0.1133526   0.00544648 -0.26315091
  0.12677361  0.15931781 -0.0288422   0.09031017 -0.22336217 -0.65368685
 -0.76088086 -0.13649197  0.04081951  0.13036691 -0.06305099 -0.07913339
  0.01366292 -0.04345531  0.        ]


In [30]:
dictionary = {'Coefficiants': model4.coef_}
avg_acc_df = pd.DataFrame(dictionary )
print(avg_acc_df)
avg_acc_df.to_csv(r'C:\Users\erics\Documents\Programme\IntroML\Task1\submission1bV4.csv', index=False, header=False)

    Coefficiants
0     -0.0876388
1     -0.7148755
2     -0.8329210
3     -0.1133526
4      0.0054465
5     -0.2631509
6      0.1267736
7      0.1593178
8     -0.0288422
9      0.0903102
10    -0.2233622
11    -0.6536869
12    -0.7608809
13    -0.1364920
14     0.0408195
15     0.1303669
16    -0.0630510
17    -0.0791334
18     0.0136629
19    -0.0434553
20     0.0000000
